#Install Dependencies

In [8]:
!pip install nltk
!git clone https://github.com/ContentSide/lingx.git

import pandas as pd
from nltk import agreement

Cloning into 'lingx'...
remote: Enumerating objects: 711, done.
remote: Counting objects: 100% (711/711), done.
remote: Compressing objects: 100% (427/427), done.
remote: Total 711 (delta 380), reused 587 (delta 261), pack-reused 0
Receiving objects: 100% (711/711), 695.82 KiB | 12.42 MiB/s, done.
Resolving deltas: 100% (380/380), done.


#Provide path to CSV file

In [2]:
path_to_csv_file = "/content/lingx/resources/TPRDB/EN-ZH_IMBst18/HumanEvaluations/errors_for_cal_kappa.csv"

#Read and Normalize the dataframe

In [3]:
df = pd.read_csv(path_to_csv_file)

df_numeral = df[['Any', 'Accuracy', 'Fluency', 
                 'Style', 'Critical','Minor']]

normalized_df=(df_numeral-df_numeral.min())/(df_numeral.max()-df_numeral.min())


normalized_df = pd.concat([
                df[['SessionSeg', 'Annotator']],
                normalized_df,
                    ],
                axis=1)
normalized_df

,SessionSeg,Annotator,Any,Accuracy,Fluency,Style,Critical,Minor
0,P00_Ist3_1,A,0.135802,0.073171,0.122449,0.000000,0.134146,0.015873
1,P00_Ist3_2,A,0.037037,0.048780,0.000000,0.000000,0.012195,0.047619
2,P00_Ist3_3,A,0.037037,0.024390,0.040816,0.000000,0.000000,0.063492
3,P00_Ist3_4,A,0.074074,0.085366,0.000000,0.000000,0.048780,0.047619
4,P00_Ist3_5,A,0.259259,0.195122,0.040816,0.181818,0.097561,0.222222
...,...,...,...,...,...,...,...,...
601,P09_Ist6_1,C,0.061728,0.073171,0.000000,0.000000,0.073171,0.000000
602,P09_Ist6_2,C,0.061728,0.073171,0.000000,0.000000,0.060976,0.015873
603,P09_Ist6_3,C,0.024691,0.036585,0.000000,0.000000,0.036585,0.000000
604,P09_Ist6_4,C,0.024691,0.036585,0.000000,0.000000,0.012195,0.031746


# Create two functions to feed `nltk.agreement.AnnotationTask` class

In [6]:
def get_annotation_task(
                        coder_column = "Annotator",
                        item_column = "SessionSeg",
                        label_column = "Any"):

    annotation_task = []
    for index, item in normalized_df.iterrows():
        annotation_task.append([item[coder_column], item[item_column], item[label_column]])

    return annotation_task

def distance_function(x,y):
    delta=x-y
    return abs(delta)


# to test the function
annotation_task = get_annotation_task(coder_column = "Annotator", item_column = "SessionSeg", label_column = "Any")

print("Sample of annotation task data:\n")
print(annotation_task[0:2])

Sample of annotation task data:

[['A', 'P00_Ist3_1', 0.13580246913580246], ['A', 'P00_Ist3_2', 0.037037037037037035]]


# Calculating Inter-Coder Agreements

In [7]:
label_column = ['Any', 'Accuracy', 'Fluency', 
                'Style', 'Critical','Minor']

def distance_function(x,y):
    delta=x-y
    return abs(delta)


for item in label_column:

    annotation_task = get_annotation_task(coder_column = "Annotator", item_column = "SessionSeg", label_column = item)
    rating_task = agreement.AnnotationTask(data=annotation_task, distance=distance_function)
    print(f"Label : {item}")
    print("kappa " +str(rating_task.kappa()))
    print("fleiss " + str(rating_task.multi_kappa()))
    print("alpha " +str(rating_task.alpha()))
    print("scotts " + str(rating_task.pi()))
    print()

Label : Any
kappa 0.8531276941717335
fleiss 0.852587431902514
alpha 0.20638351404601274
scotts 0.8516922256612892

Label : Accuracy
kappa 0.9154301335633132
fleiss 0.9152220922178368
alpha 0.44094316795496924
scotts 0.9149690020851614

Label : Fluency
kappa 0.8166101733187144
fleiss 0.814770539088119
alpha 0.009221563025255008
scotts 0.801914464606351

Label : Style
kappa 0.741835289161885
fleiss 0.7385729115695417
alpha 0.021035598705502312
scotts 0.7351218608099348

Label : Critical
kappa 0.910536990343188
fleiss 0.9104056677390059
alpha 0.4276497553737072
scotts 0.9101906242216101

Label : Minor
kappa 0.8709120127802246
fleiss 0.8659092126789742
alpha -0.03511052376349788
scotts 0.862036583884514

